In [1]:
import numpy as np
import numpy_financial as npf

# Question 1 [16 marks]

Key figures

In [2]:
equity_risk_prem = 0.0482
risk_free_ROI = 0.0425
market_debt_val = 900e6
market_equity_val = 2.4e9
equity_b = 1.3
pre_tax_debt_cost = 0.0925
corp_tax = 0.375
debt_investment = 80e6
equity_investment = 20e6

### a) Calculate the weighted average cost of capital of ABC prior to its new project investment. [3 marks] 

The WACC is calculated using the following formula:

\begin{equation}
\text{WACC} = \frac{\text{expected operating income}}{\text{market value of all securities}}
\end{equation}

Where the operating income is the sum of all operating incomes of ABC company and the market value of all securities is the total of the market value of all securities.

In [3]:
WACC = (market_debt_val * pre_tax_debt_cost + market_equity_val * equity_risk_prem) / (market_debt_val + market_equity_val)
print(f'Weighted Average Cost of Capital = {WACC:.4f}')

Weighted Average Cost of Capital = 0.0603


### b) Find ABC’s asset beta prior to the new project. [2 marks]

We use the formula:
$$
\beta_A = \frac{(\beta_D.D + \beta_E.E)}{V}
$$

where $V = D+E$ is the total value of the securities of the firm

$D$ is the debt portion of the firm, and $E$ is the equity portion. Where we also have that $\beta_D$ is the debt $\beta$, and $\beta_E$ is the equity $\beta$


In [4]:
beta_A = equity_b * market_equity_val / (market_debt_val + market_equity_val)
print(f'Asset Beta = {beta_A:.4f}')

Asset Beta = 0.9455


### c) If the new project has the same asset beta as the ABC company in b), find the project equity beta. [2 marks]

In [5]:
D = market_debt_val + debt_investment
E = market_equity_val + equity_investment
beta_E_proj = beta_A * (1 + D / (E + D))
print(f'Project equity beta = {beta_E_proj:.4f}')

Project equity beta = 1.2180


### d) Use python functions to automatically calculate project FCFs and prove that the aftertax free cash flow generated for the next three years are \\$ 48 million, \\$ 52 million, and \\$ 54.4 million, respectively. [2 marks]

#### FCF = (Revenues – Operating Costs – Depreciation) * (1 – Tax Rate) + Depreciation


In [6]:
def FCF(rev, op, dep, tax):
    return (rev - op - dep) * (1 - tax) + dep

In [7]:
rev = [99.2, 109.6, 115.44]
ops = [32, 36, 38]
deps = [16, 16, 16]
FCFs = []

for t, (r, o, d) in enumerate(zip(rev, ops, deps)):
    FCFs.append(FCF(r, o, d, corp_tax))
    print(f'FCF for year {t + 1} = ${FCF(r, o, d, corp_tax):.1f} million')

FCF for year 1 = $48.0 million
FCF for year 2 = $52.0 million
FCF for year 3 = $54.4 million


### e) Find the project NPV and IRR with the next three years after-tax free cash flow given in. [2 marks]

To find the IRR we create the following function.
Brief description here please.

In [8]:
def IRR_Step(start, step, thresh=0.0001):
    r = start
    npv = lambda x: FCFs[0] / (1 + x) + CFs[1] / ((1 + x) ** 2) + CFs[2] / ((1 + x) ** 3) - 100
    guess = npv(r)
    while abs(guess) > thresh:
        if guess > 0:
            r += step
            new_guess = npv(r)
            if new_guess < 0:
                step /= 2
                r -= step
                guess = npv(r)
            else:
                guess = new_guess
        if guess < 0:
            r -= step
            new_guess = npv(r)
            if new_guess > 0:
                step /= 2
                r += step
                guess = npv(r)
            else:
                guess = new_guess

    return r

In [24]:
# calculated using our own funcitons

CFs = [48,52,54.4]
NPV =  FCFs[0] / (1 + risk_free_ROI) + CFs[1] / ((1 + risk_free_ROI) ** 2) + CFs[2] / ((1 + risk_free_ROI) ** 3) - 100
print(f'NPV = {NPV}')
IRR = IRR_Step(0.5, 0.01, thresh=0.00001)
print(f'IRR = {IRR:.5f}')

NPV = 41.90409210164432
IRR = 0.24691


In [25]:
# calculated using npf

CFs = [-100,48,52,54.4]

NPV = npf.npv(risk_free_ROI,CFs)
print(f'NPV = {NPV}')

IRR = npf.irr(CF)
print(f'IRR = {IRR:.5f}')

NPV = 41.904092101644316
IRR = 0.24691


### f) Use the discount dividend method and find the current value of company DEF stock under both economic conditions. [3 marks]

We use the formula:

$\begin{aligned} P_{0} &=\sum_{t=1}^{H} \frac{D_{t}}{(1+r)^{t}}+\frac{P_{H}}{(1+r)^{H}} \\ &=\sum_{t=1}^{\infty} \frac{D_{t}}{(1+r)^{t}} \end{aligned}$

where $P_i$ is the price after $i$ time periods, $H$ is the horizon, $D_t$ is the dividend at the end of time period $t$, and $r$ is the cost of equity capital

In [26]:
#Under constant growth - k=7.35%, g=3.5%
P0_D1_035 = 0.65 * 1.035 / (0.0735 - 0.035)
print(f'Present Value under economic fall = {P0_D1_035:.3f}')

#Under boom
P0_D1_12 = 0.65 * 1.12 / (0.0735 - 0.12) 
P0_D5_12 = 0.65 * (1.12 ** 4) / (0.0735 - 0.12)
P0_D5_035 = 0.65 * (1.12 ** 4) / (0.0735 - 0.035)

P0 = P0_D1_12 - P0_D5_12 + P0_D5_035
print(f'Present Value under economic boom = {P0:.3f}')

Present Value under economic fall = 17.474
Present Value under economic boom = 32.905


### g) Calculate GHI company’s sustainable growth rate, find the value of the company’s stock at the beginning of 2022 and determine the company’s present value of growth opportunities. [2 marks]

The sustainable growth rate, $g$, is calculated using:
$$
g = b \times \text{ROE}
$$
where $b$ is the plowback ratio and is also known as the Earnings Retention Ratio, and $\text{ROE}$ is the Return on Equity.

The current price of the stuck is calculated with:
$$
P_0 = \frac{D_0(1+g)}{r-g}
$$
where $r$ is the market capitalisation rate, $D_0$ is the current dividend, and $g$ is the sustainable growth rate we just found.

We then calculate the company's present value of growth using:
$$
\text{PVGO}= P_0 - \frac{E_1}{r}
$$
where $E_1$ = the earnings per share (EPS) during time period 1, $P_0$ is the current price, and $r$ is the market capitalisation rate.

In [11]:
ERR = 0.6
EPS = 2
Payout_ratio = 0.4 # = dividends / eps
ROE = 0.14
g_sus = ERR * ROE
dividends = Payout_ratio * EPS
P = dividends * (1 + g_sus) / (0.11 - g_sus)
PVGO = P - (EPS / 0.11) #P0 - E1/r

print(f'Sustainable Growth Rate = {g_sus:.3f}', f'Stock Value (2022) = {P:.2f}', f'PVGO = {PVGO:.3f}', sep='\n')

Sustainable Growth Rate = 0.084
Stock Value (2022) = 33.35
PVGO = 15.172


In [4]:
! git push

[main 3a1c2aa] updates
 3 files changed, 421 insertions(+), 73 deletions(-)


The file will have its original line endings in your working directory
The file will have its original line endings in your working directory
The file will have its original line endings in your working directory
The file will have its original line endings in your working directory
The file will have its original line endings in your working directory
The file will have its original line endings in your working directory
